In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
import tqdm

In [24]:
train = pd.read_csv('datasets/train.csv')
test = pd.read_csv('datasets/test.csv')
original = pd.read_csv('datasets/credit_risk_dataset.csv').dropna()

# unneccessary column repeated in index
train.drop(columns='id',inplace=True)
test.drop(columns='id',inplace=True)

# straight-up one hot encoding
train = pd.get_dummies(train)
test = pd.get_dummies(test)
original = pd.get_dummies(original)

# just to see whether the results get better with a merge
train = pd.concat([train,original],axis=0)

In [25]:
X = train.loc[:,train.columns!='loan_status']
y = train[['loan_status']]

No reason to do train, test split to be honest. I will recieve results from Kaggle anyway.

In [4]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [26]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [30]:
# Convert data to tensors
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y.values, dtype=torch.float32)
# print(y.shape)
# print(y.view(-1,1).shape)
# torch.Size([91226, 1])
# torch.Size([91226, 1])

In [31]:
X.shape

torch.Size([87283, 26])

In [37]:
# Create TensorDataset and DataLoader
dataset = TensorDataset(X, y)

loader = DataLoader(dataset, batch_size=16, shuffle=True) # smaller batchsize generalizes better

# Define a simple neural network

model = nn.Sequential(
    nn.Linear(X.shape[1],32),
    nn.ReLU(),
    nn.Linear(32,16),
    nn.ReLU(),
    nn.Linear(16,1),
    nn.Sigmoid()
)

In [38]:
criterion = nn.BCELoss()  # Binary Cross-Entropy loss
optimizer = optim.Adam(model.parameters(), lr=0.001) 

num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    
    for features, targets in loader:
        # Forward pass
        outputs = model(features)
        loss = criterion(outputs, targets)  # Loss calculation
        
        # Backward pass and optimization
        optimizer.zero_grad()  # Zero gradients
        loss.backward()        # Backpropagation
        optimizer.step()       # Update weights

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

In [83]:
predictions = model(torch.tensor(scaler.transform(test.values),dtype=torch.float32))

/home/aner/.venvs/energy/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [86]:
id = pd.read_csv('test.csv').id

In [91]:
predictions.squeeze(-1)

tensor([1.0000, 0.0421, 0.2187,  ..., 0.0309, 0.2029, 1.0000],
       grad_fn=<SqueezeBackward1>)

In [89]:
id.shape

(39098,)

In [93]:
df_submissions = pd.DataFrame({'id':id,'loan_status':predictions.squeeze(-1).detach().numpy()})

In [96]:
df_submissions

,id,loan_status
0,58645,1.000000
1,58646,0.042138
2,58647,0.218736
3,58648,0.049134
4,58649,0.202852
...,...,...
39093,97738,0.053374
39094,97739,0.012651
39095,97740,0.030900
39096,97741,0.202852


In [97]:
df_submissions.to_csv("submission.csv",index=False)